In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from math import radians
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from math import radians
import pandas as pd
from datetime import datetime, timedelta
import pandas as pd
from datetime import datetime
from geopy.distance import geodesic

In [ ]:
train_data = pd.read_parquet('train.parquet')
test_data = pd.read_parquet('test.parquet')
final_test_data = pd.read_parquet('final_test.parquet')


bike_parking_data = pd.read_excel('stationnement-velo-en-ile-de-france.xlsx')
historical_data = pd.read_csv('comptage-velo-donnees-sites-comptage_2020.csv', sep=';')
velib_data = pd.read_excel('velib-disponibilite-en-temps-reel.xlsx')



### Cleaning Historical Data to Extend Our Datase

In [13]:
# Convert 'Date et heure de comptage' to datetime with the correct timezone offset handling
historical_data['Date et heure de comptage'] = pd.to_datetime(historical_data['Date et heure de comptage'], utc=True)

# Define the cutoff date also in UTC for consistency
cutoff_date = pd.Timestamp('2020-09-01 01:00:00', tz='UTC')

# Filter the data
historical_data = historical_data[historical_data['Date et heure de comptage'] < cutoff_date]

# Drop columns by name
historical_data.drop(
    ['Identifiant du point de comptage', 
     'Date d\'installation du point de comptage', 
     'Lien vers photo du point de comptage', 
     'Coordonnées géographiques'], 
    axis=1, inplace=True)

# Rename the remaining columns
historical_data.rename(
    columns={
        'Nom du point de comptage': 'counter_name',
        'Comptage horaire': 'bike_count',
        'Date et heure de comptage': 'date'
    }, inplace=True)

# Create a new column for the logarithmic bike count
historical_data['log_bike_count'] = np.log(historical_data['bike_count'] + 1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1503173 entries, 18 to 2314578
Data columns (total 7 columns):
 #   Column                                    Non-Null Count    Dtype              
---  ------                                    --------------    -----              
 0   Identifiant du point de comptage          1503173 non-null  int64              
 1   Nom du point de comptage                  1479741 non-null  object             
 2   Comptage horaire                          1503173 non-null  int64              
 3   Date et heure de comptage                 1503173 non-null  datetime64[ns, UTC]
 4   Date d'installation du point de comptage  1479741 non-null  object             
 5   Lien vers photo du point de comptage      1479741 non-null  object             
 6   Coordonnées géographiques                 1479741 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(2), object(4)
memory usage: 91.7+ MB


In [16]:
historical_data.tail()

,counter_name,bike_count,date,log_bike_count
2314574,90 Rue De Sèvres,3,2020-08-31 00:00:00+00:00,1.386294
2314575,90 Rue De Sèvres,1,2020-08-31 03:00:00+00:00,0.693147
2314576,90 Rue De Sèvres,10,2020-08-31 05:00:00+00:00,2.397895
2314577,90 Rue De Sèvres,40,2020-08-31 12:45:00+00:00,3.713572
2314578,90 Rue De Sèvres,5,2020-08-31 23:30:00+00:00,1.791759


In [17]:
# Saving historical data to a new csv file
historical_data.to_csv("historical_data_2020.csv", index=False)

## Adding Velib Availability within Threshold Distance from Counters

In [9]:
# Prepare Velib station data
# Assuming 'coordonnee geographique' column contains the lat-lon information
velib_data[['latitude', 'longitude']] = velib_data['Coordonnées géographiques'].str.split(',', expand=True).astype(float)
velib_lat_lon = np.deg2rad(velib_data[['latitude', 'longitude']].values)

# Set threshold for a 5-minute walking distance (approximately 0.42 km)
walking_distance_km = 0.42

# Function to calculate distances using BallTree
def calculate_distances(lat, lon, velib_lat_lon):
    tree = BallTree(velib_lat_lon, metric='haversine')
    distances, _ = tree.query(np.array([[lat, lon]]), k=len(velib_lat_lon))
    return distances[0]

# Extract unique latitude and longitude pairs from combined data
unique_coords = combined_data[['latitude', 'longitude']].drop_duplicates()

# Calculate Velib capacities within walking distance for each unique coordinate
velib_capacities = {}
for _, row in unique_coords.iterrows():
    rad_lat, rad_lon = np.deg2rad([row['latitude'], row['longitude']])
    distances = calculate_distances(rad_lat, rad_lon, velib_lat_lon)
    velib_capacities[(row['latitude'], row['longitude'])] = velib_data[distances <= walking_distance_km / 6371]['Capacité de la station'].sum()

# Map these capacities back to the combined data
combined_data['number_of_Velibs_within_5min_walk'] = combined_data.apply(
    lambda row: velib_capacities[(row['latitude'], row['longitude'])],
    axis=1
)

# Select only the date column and the new Velib capacity column
columns_to_save = ['date', 'number_of_Velibs_within_5min_walk']
data_to_save = combined_data[columns_to_save]

# Save the selected data to a CSV file
data_to_save.to_csv('number_of_velibs_nearby.csv', index=False)

# Display the first few rows of the data to save
print(data_to_save.head())

                 date  number_of_Velibs_within_5min_walk
0 2020-09-01 02:00:00                                221
1 2020-09-01 03:00:00                                221
2 2020-09-01 04:00:00                                221
3 2020-09-01 15:00:00                                221
4 2020-09-01 18:00:00                                221


## Creating a Paris School Holiday Dataset

In [11]:
# Define the school holiday periods for Paris (Area C) from 2020 to 2022
school_holidays = [
    {'start': '2020-02-08', 'end': '2020-02-23'},  # Example holiday dates
    {'start': '2020-04-04', 'end': '2020-04-20'},
    {'start': '2020-07-06', 'end': '2020-09-01'},
    {'start': '2020-10-17', 'end': '2020-11-02'},    
    {'start': '2020-12-19', 'end': '2021-01-04'},
    
    {'start': '2021-02-13', 'end': '2021-03-01'},
    {'start': '2021-04-17', 'end': '2021-05-03'},  # Example holiday dates
    {'start': '2021-07-06', 'end': '2021-09-02'},
    {'start': '2021-12-18', 'end': '2022-01-03'},
]

# Convert the dates to datetime objects
for holiday in school_holidays:
    holiday['start'] = datetime.strptime(holiday['start'], '%Y-%m-%d')
    holiday['end'] = datetime.strptime(holiday['end'], '%Y-%m-%d')

# Generate a date range from 2020 to 2022, including every hour
start_date = datetime(2020, 1, 1, 0, 0)
end_date = datetime(2022, 12, 31, 23, 59)
date_range = pd.date_range(start=start_date, end=end_date, freq='H')

# Create a DataFrame
df = pd.DataFrame(date_range, columns=['datetime'])

# Function to check if a date is a school holiday
def is_school_holiday(date):
    for holiday in school_holidays:
        if holiday['start'] <= date <= holiday['end']:
            return 1
    return 0

# Apply the function to each date
df['is_school_holiday'] = df['datetime'].apply(is_school_holiday)

# Save the DataFrame to a CSV file
csv_file_path = 'paris_school_holidays_2020_2022_correct.csv'
df.to_csv(csv_file_path, index=False)

csv_file_path

'paris_school_holidays_2020_2022_correct.csv'

## Extracting and Scaling Velib Availability Data from External Dataset for Bike Demand Prediction

In [6]:
# Function to extract latitude and longitude from a 'Geo Point' column
def extract_lat_lon(geo_point):
    lat, lon = map(float, geo_point.split(','))
    return lat, lon

# Function to calculate the distance between two points
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

# Function to process the bike parking dataset
def process_bike_parking_dataset(bike_parking_data):
    # Convert 'date_modif' column to datetime format and remove timezone information
    bike_parking_data['date_modif'] = pd.to_datetime(bike_parking_data['date_modif'].astype(str).str[:-6], errors='coerce')

    # Define the date range
    start_date = datetime(2020, 2, 1)
    end_date = datetime(2021, 12, 1)

    # Filter the data to include only entries within the specified date range
    filtered_df = bike_parking_data[(bike_parking_data['date_modif'] >= start_date) & (bike_parking_data['date_modif'] <= end_date)].copy()

    # Extract latitude and longitude
    lat_lon = filtered_df['Geo Point'].apply(extract_lat_lon)
    filtered_df['latitude'] = lat_lon.apply(lambda x: x[0])
    filtered_df['longitude'] = lat_lon.apply(lambda x: x[1])

    return filtered_df


# Process the bike parking dataset
processed_bike_parking_df = process_bike_parking_dataset(bike_parking_data)

# Define a biking distance threshold (in kilometers)
biking_distance_threshold = 2.0

# Initialize an empty dictionary to hold the results
counter_bike_parking_capacity = {}

# Iterate over each counter in test_data
for index, counter in test_data.iterrows():
    counter_name = counter['counter_name']
    counter_lat = counter['latitude']
    counter_lon = counter['longitude']

    # Calculate the distance to each parking spot and sum the capacities if within biking distance
    total_capacity = 0
    for _, parking in processed_bike_parking_df.iterrows():
        parking_lat = parking['latitude']
        parking_lon = parking['longitude']
        distance = calculate_distance(counter_lat, counter_lon, parking_lat, parking_lon)

        # Check if the parking is within the biking distance
        if distance <= biking_distance_threshold:
            # Scale the capacity by distance and add to the total
            scaled_capacity = parking['capacite'] * (1 - distance / biking_distance_threshold)
            total_capacity += scaled_capacity

    # Store the result
    counter_bike_parking_capacity[counter_name] = total_capacity

# Convert the results to a DataFrame
result_df = pd.DataFrame(list(counter_bike_parking_capacity.items()), columns=['Counter_Name', 'Bike_Parking_Capacity'])

csv_file_path = 'counter_bike_parking_capacity.csv'
result_df.to_csv(csv_file_path, index=False)

# Display the resulting DataFrame
result_df.head()


KeyboardInterrupt: 

In [ ]:
# Function to extract latitude and longitude from a 'Geo Point' column
def extract_lat_lon(geo_point):
    return list(map(float, geo_point.split(',')))

# Function to calculate the distance between two points
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

# Function to process the bike parking dataset
def process_bike_parking_dataset(bike_parking_data):
    # Convert 'date_modif' column to datetime format and remove timezone information
    bike_parking_data['date_modif'] = pd.to_datetime(bike_parking_data['date_modif'].astype(str).str[:-6], errors='coerce')

    # Define the date range
    start_date = datetime(2020, 2, 1)
    end_date = datetime(2021, 12, 1)

    # Filter the data to include only entries within the specified date range
    filtered_df = bike_parking_data[(bike_parking_data['date_modif'] >= start_date) & (bike_parking_data['date_modif'] <= end_date)]

    # Extract latitude and longitude
    filtered_df[['latitude', 'longitude']] = pd.DataFrame(filtered_df['Geo Point'].apply(extract_lat_lon).tolist(), index=filtered_df.index)

    return filtered_df

# Load the smaller dataset
final_test_data = pd.read_parquet('final_test.parquet')

# Process the bike parking dataset
processed_bike_parking_df = process_bike_parking_dataset(bike_parking_data)

# Define a biking distance threshold (in kilometers)
biking_distance_threshold = 2.0

# Vectorized distance calculation
def calculate_scaled_capacity(row, bike_parking_df, threshold):
    distances = bike_parking_df.apply(lambda x: calculate_distance(row['latitude'], row['longitude'], x['latitude'], x['longitude']), axis=1)
    close_parkings = bike_parking_df[distances <= threshold]
    scaled_capacities = close_parkings['capacite'] * (1 - distances[distances <= threshold] / threshold)
    return scaled_capacities.sum()

# Apply the calculation to each row in the final_test_data
result_df = final_test_data.copy()
result_df['Bike_Parking_Capacity'] = result_df.apply(calculate_scaled_capacity, args=(processed_bike_parking_df, biking_distance_threshold), axis=1)

# Save the results
csv_file_path = 'counter_bike_parking_capacity_optimized.csv'
result_df.to_csv(csv_file_path, index=False)

# Display the resulting DataFrame
result_df.head()